In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/cs679_project_compare_40/code/compare_graphs/numeric_compare/wavs/seturaj_files

/content/drive/MyDrive/cs679_project_compare_40/code/compare_graphs/numeric_compare/wavs/seturaj_files


In [ ]:
!pip install librosa pesq jiwer

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 5.8 MB/s eta 0:00:00
  Created wheel for pesq: filename=pesq-0.0.4-cp310-cp310-linux_x86_64.whl size=262960 sha256=0e776f0f2418c9327a24f131f003cb1caa7163fd3b295c79d1e6b2862eff8b65
  Stored in directory: /root/.cache/pip/wheels/c5/4e/2c/251524370c0fdd659e99639a0fbd0ca5a782c3aafcd456b28d
Successfully built pesq


In [ ]:
import librosa
import numpy as np

def compute_mcd(file1, file2):
    y1, sr1 = librosa.load(file1, sr=None)
    y2, sr2 = librosa.load(file2, sr=None)

    # Ensure same sample rate for both files
    if sr1 != sr2:
        y2 = librosa.resample(y2, orig_sr=sr2, target_sr=sr1)

    mfcc1 = librosa.feature.mfcc(y=y1, sr=sr1)
    mfcc2 = librosa.feature.mfcc(y=y2, sr=sr1)

    # if shape is different
    if mfcc1.shape[1] > mfcc2.shape[1]:
        mfcc1 = mfcc1[:, :mfcc2.shape[1]]
    elif mfcc2.shape[1] > mfcc1.shape[1]:
        mfcc2 = mfcc2[:, :mfcc1.shape[1]]

    # Compute the Euclidean distance between the two MFCC feature matrices
    mcd_value = np.mean(np.sqrt(np.sum((mfcc1 - mfcc2) ** 2, axis=0)))
    return mcd_value


In [ ]:
from pesq import pesq

def compute_pesq(reference_file, degraded_file, fs):
    """
    Compute the PESQ score.
    fs: sampling rate, use 16000 for wideband or 8000 for narrowband.
    """
    ref, _ = librosa.load(reference_file, sr=fs)
    deg, _ = librosa.load(degraded_file, sr=fs)
    score = pesq(fs, ref, deg, 'wb' if fs == 16000 else 'nb')
    return score


In [ ]:
from jiwer import wer

def compute_wer(transcription_original, transcription_transferred):
    error_rate = wer(transcription_original, transcription_transferred)
    return error_rate


In [ ]:
!pip install pystoi

In [ ]:
import soundfile as sf
from pystoi.stoi import stoi

def calculate_stoi(original_path, transferred_path):
    # Load the original and transferred speech files
    original_signal, fs_orig = sf.read(original_path)
    transferred_signal, fs_trans = sf.read(transferred_path)

    # Ensure both files have the same sampling rate
    if fs_orig != fs_trans:
        transferred_signal = librosa.resample(transferred_signal, orig_sr=fs_trans, target_sr=fs_orig)
        # raise ValueError("Sampling rates do not match. STOI requires both signals to have the same sampling rate.")

    # Check if both signals are of the same length; pad with zeros if not
    if len(original_signal) != len(transferred_signal):
        max_len = max(len(original_signal), len(transferred_signal))
        original_signal = np.pad(original_signal, (0, max_len - len(original_signal)), 'constant')
        transferred_signal = np.pad(transferred_signal, (0, max_len - len(transferred_signal)), 'constant')

    # Calculate the STOI score
    stoi_score = stoi(original_signal, transferred_signal, fs_orig, extended=False)

    return stoi_score

In [ ]:
import soundfile as sf
def calculate_mse(original_path, transferred_path):
    # Load the original and transferred speech files
    original_signal, fs_orig = sf.read(original_path)
    transferred_signal, fs_trans = sf.read(transferred_path)

    # Ensure both files have the same sampling rate
    if fs_orig != fs_trans:
        transferred_signal = librosa.resample(transferred_signal, orig_sr=fs_trans, target_sr=fs_orig)

    # Check if both signals are of the same length; pad with zeros if not
    if len(original_signal) != len(transferred_signal):
        max_len = max(len(original_signal), len(transferred_signal))
        original_signal = np.pad(original_signal, (0, max_len - len(original_signal)), 'constant')
        transferred_signal = np.pad(transferred_signal, (0, max_len - len(transferred_signal)), 'constant')

    # Calculate the Mean Squared Error
    mse = np.mean((original_signal - transferred_signal) ** 2)

    return mse

In [ ]:
!pip install pyAudioAnalysis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 22.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyAudioAnalysis: filename=pyAudioAnalysis-0.3.14-py3-none-any.whl size=41264373 sha256=b993f93378b4e697d5cbdb5144aaf2853a518bcff903837b995389dd593ee524
  Stored in directory: /root/.cache/pip/wheels/a7/54/73/fa830689c2440d2c81ff175c60e374930ad1607a8881e0f43f
Successfully built pyAudioAnalysis


# Methods

**MCD**: Mel-Cepstral Distortion (MCD)

**PESQ**: Perceptual Evaluation of Speech Quality (PESQ)

**WER**: Word Error Rate (WER)

**STOI**: The Short-Time Objective Intelligibility (STOI)

**MSE**: Mean Square Error

## Description

**MCD** is a measure of the spectral difference between two audio signals, typically used to assess the quality of voice conversion systems. Lower values indicate higher similarity between the audio samples.


**PESQ** scores range from -0.5 to 4.5, where higher scores indicate better speech quality. Scores above 3 are generally considered good, indicating high perceptual quality, while scores below 2 suggest poor quality, with noticeable distortions or degradation.


**WER** is a measure of the transcription error rate, calculated as the ratio of the number of errors (insertions, deletions, substitutions) to the total number of words spoken. WER values range from 0 to 1 (or 0% to 100%), where 0 indicates perfect transcription with no errors, and higher values indicate more errors.


**STOI** measures the intelligibility of speech signals and is particularly useful in evaluating the clarity of speech after style transfer. Compute the STOI score between the original and the style-transferred speech. Higher scores indicate better intelligibility.

In practical terms:

Above 0.75: Generally considered good intelligibility. Listeners are likely to understand the speech without significant effort.
0.6 to 0.75: Moderate intelligibility. While listeners can understand the speech, they might need to pay more attention, and some parts of the speech might be harder to comprehend.
Below 0.6: Poor intelligibility. Listeners may struggle to understand the speech, and there could be significant distortion or loss of information.

**MSE** quantifies the average squared difference between the values of the original and transferred signals, essentially measuring the variance between these two datasets

# Make a table

In [ ]:
import pandas as pd
def main(original_file, transferred_file):
    # Compute MCD
    mcd_value = compute_mcd(original_file, transferred_file)

    # Compute PESQ
    pesq_score = compute_pesq(original_file, transferred_file, 16000)

    # STOI
    stoi_score = calculate_stoi(original_file, transferred_file)

    # MSE
    mse_score = calculate_mse(original_file, transferred_file)

    return {
        "MCD": mcd_value,
        "PESQ": pesq_score,
        "STOI": stoi_score,
        "MSE": mse_score
    }

def evaluate_files(original_file, transferred_files, row_names):
    results = []

    for i in range(len(transferred_files)):
        metrics = main(original_file, transferred_files[i])
        metrics['Speaker Encoder'] = row_names[i]  # Add the file name for reference
        results.append(metrics)

    # Create a DataFrame
    df = pd.DataFrame(results)
    df.set_index('Speaker Encoder', inplace=True)
    return df

In [ ]:
# original_files = ['male_original_dmz.wav','male_original_two_old_friend.wav','male_original_2016.wav']
# transferred_files = ['p256_male_transfered_dmz.wav','p228_female_transfered_two_old_friend.wav','p256male_transfered_2016.wav']
# for i in range(3):
#     print(main(original_files[i], transferred_files[i]))

In [ ]:
file_location = ''
original_file = 'male_original_two_old_friend.wav'
transferred_files = [file_location+'converted_uttr_chinese_male.wav',
                     file_location+'converted_uttr_chinese_female.wav']
for i in range(2):
    print(main(original_file, transferred_files[i]))

{'MCD': 77.51221, 'PESQ': 1.1056644916534424, 'STOI': 0.602104520305502, 'MSE': 0.0023813346500537592}
{'MCD': 74.97293, 'PESQ': 1.1159816980361938, 'STOI': 0.6197005415150998, 'MSE': 0.002193843786517391}


In [ ]:
dict1 = {'MCD': 77.51221, 'PESQ': 1.1056644916534424, 'STOI': 0.602104520305502, 'MSE': 0.0023813346500537592}
dict2 = {'MCD': 74.97293, 'PESQ': 1.1159816980361938, 'STOI': 0.6197005415150998, 'MSE': 0.002193843786517391}

# Convert dictionaries to DataFrame
df1 = pd.DataFrame([dict1])
df2 = pd.DataFrame([dict2])

# Concatenate the DataFrames
result_df = pd.concat([df1, df2], ignore_index=True)
result_df.index = ['Male_Voice', 'Female_Voice']
result_df

,MCD,PESQ,STOI,MSE
Male_Voice,77.51221,1.105664,0.602105,0.002381
Female_Voice,74.97293,1.115982,0.619701,0.002194


In [ ]:
data = {
    "MCD": [190.872009, 188.664474, 249.722000, 169.612961],
    "PESQ": [1.095009, 1.057051, 1.046293, 1.097576],
    "STOI": [0.160352, 0.175409, 0.056638, 0.576549],
    "MSE": [0.031352, 0.024982, 0.018245, 0.020387]
}

# Create the DataFrame
df = pd.DataFrame(data, index=["Zero-Shot", "Facebook", "SpeechBrain", "ConVGRU"])

# Convert to LaTeX
latex_code = df.to_latex()

print(latex_code)

\begin{tabular}{lrrrr}
\toprule
 & MCD & PESQ & STOI & MSE \\
\midrule
Zero-Shot & 190.872009 & 1.095009 & 0.160352 & 0.031352 \\
Facebook & 188.664474 & 1.057051 & 0.175409 & 0.024982 \\
SpeechBrain & 249.722000 & 1.046293 & 0.056638 & 0.018245 \\
ConVGRU & 169.612961 & 1.097576 & 0.576549 & 0.020387 \\
\bottomrule
\end{tabular}



$$
\begin{tabular}{lrrrr}
\toprule
 & MCD & PESQ & STOI & MSE \\
\midrule
Zero-Shot & 190.872009 & 1.095009 & 0.160352 & 0.031352 \\
Facebook & 188.664474 & 1.057051 & 0.175409 & 0.024982 \\
SpeechBrain & 249.722000 & 1.046293 & 0.056638 & 0.018245 \\
ConVGRU & 169.612961 & 1.097576 & 0.576549 & 0.020387 \\
\bottomrule
\end{tabular}
$$

In [ ]:
break

# Below are combination of above

In [ ]:
def main(original_file, transferred_file):
    # Compute MCD
    mcd_value = compute_mcd(original_file, transferred_file)
    print(f"MCD: {mcd_value}")

    # Compute PESQ
    # Replace '16000' with '8000' if your audio is narrowband.
    pesq_score = compute_pesq(original_file, transferred_file, 16000)
    print(f"PESQ: {pesq_score}")

    # STOI
    stoi_score = calculate_stoi(original_file, transferred_file)
    print(f"STOI score: {stoi_score}")

    # MSE
    mse_score = calculate_mse(original_file, transferred_file)
    print(f"MSE score: {mse_score}")
    return [mcd_value, pesq_score, stoi_score, mse_score]


In [ ]:
main('wavs/p256_003/p256_003.wav', 'wavs/p256_003/p225xp256.wav')

MCD: 181.4127197265625
PESQ: 1.0934630632400513
STOI score: 0.1817480358493922
MSE score: 0.01434925263550775


[181.41272, 1.0934630632400513, 0.1817480358493922, 0.01434925263550775]

In [ ]:
main('wavs/p228_003/p228_003.wav', 'wavs/p228_003/p225xp228.wav')

MCD: 184.4299774169922
PESQ: 2.2971787452697754
STOI score: 0.24639579933468386
MSE score: 0.010027609887696654


[184.42998, 2.2971787452697754, 0.24639579933468386, 0.010027609887696654]

In [ ]:
main('wavs/p225_003/p225_003.wav', 'wavs/p225_003/p225xp228.wav')

MCD: 106.23403930664062
PESQ: 1.1598138809204102
STOI score: 0.7631630574028471
MSE score: 0.02743471773271386


[106.23404, 1.1598138809204102, 0.7631630574028471, 0.02743471773271386]

In [ ]:
main('wavs/p225_003/p225_003.wav', 'wavs/p225_003/p225xp225.wav')

MCD: 106.23150634765625
PESQ: 1.2678604125976562
STOI score: 0.7820360269488631
MSE score: 0.026854449864839872


[106.23151, 1.2678604125976562, 0.7820360269488631, 0.026854449864839872]

In [ ]:
main('wavs/p225_001/p225_001.wav', 'wavs/p225_001/p225xp225.wav')

MCD: 182.07955932617188
PESQ: 1.0715758800506592
STOI score: 0.17878489325197752
MSE score: 0.024701175366913644


[182.07956, 1.0715758800506592, 0.17878489325197752, 0.024701175366913644]